#**Imports**

In [ ]:
#@title Install HDBSCAN and UMAP

!pip install uv

try:
    import hdbscan
    import umap
except ImportError:
    !uv pip install --system hdbscan
    !uv pip install --system umap-learn
    import hdbscan
    import umap

In [ ]:
#@title Import libraries

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import itertools



def check_for_nans(df, df_name):
    """
    Checks the given DataFrame for NaN values and prints the count for each column containing NaNs.

    Args:
    df (pd.DataFrame): DataFrame to be checked for NaN values.
    df_name (str): The name of the DataFrame as a string, used for printing.
    """
    # Check if the DataFrame has any NaN values and print a warning if it does.
    nan_columns = df.columns[df.isna().any()].tolist()

    if nan_columns:
        for col in nan_columns:
            nan_count = df[col].isna().sum()
            print(f"Column '{col}' in {df_name} contains {nan_count} NaN values.")
    else:
        print(f"No NaN values found in {df_name}.")

#**Load your data and pre-process**

In [ ]:
#@title Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title Load your data folder

#@markdown ### Select path to experiment folder
Data_folder = r"/content/gdrive/MyDrive/" #@param {type:"string"}

import os
filenames_ = os.listdir(Data_folder)

#@markdown ### Setup rule to select the right files in your folder (currently, match a substring in the filename of each imge, e.g. file extension)
import re as regex

# Regex pattern to match files that end with .ome.tiff
regex_pattern = ".csv" #@param {type:"string"}

# Filtering the list using the regex
filenames_filtered = sorted([f for f in filenames_ if regex.search(regex_pattern, f)])
filenames_filtered

In [ ]:
#@title Concatenate btrack files and assign genotype

import pandas as pd

btrack_list_of_dfs = []

#@markdown ### Setup rule(s) to assign genotype/condition (must match a substring in the filename of each image) - repeat as much as needed
conditon_1 = "condition_1" #@param {type:"string"}
conditon_2 = "condition_2" #@param {type:"string"}

for fname in filenames_filtered:
    btrack_csv_to_df = pd.read_csv(os.path.join(Data_folder, fname))
    if conditon_1 in fname:
      btrack_csv_to_df['Condition'] = conditon_1
    elif conditon_2 in fname:
      btrack_csv_to_df['Condition'] = conditon_2
    with pd.option_context("display.max_columns", None
                          ):
        with pd.option_context("display.max_rows", 2):
            display(btrack_csv_to_df)
    print(btrack_csv_to_df.shape)
    btrack_list_of_dfs.append(btrack_csv_to_df)

btrack_df = pd.concat(btrack_list_of_dfs, ignore_index=True)

print(btrack_df.shape)
print(btrack_df.columns)

with pd.option_context("display.max_columns", None):
    with pd.option_context("display.max_rows", 6):
        display(btrack_df)

In [ ]:
#@title Add and adjust features / columns

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# Add sample ID column
btrack_df['sample_id_numeric'] = le.fit_transform(btrack_df.sample_id.values)
btrack_df['sample_id_numeric'] = (btrack_df['sample_id_numeric'] + 101) * 100
# Add unique_ID for each track
btrack_df['ID'] = btrack_df['ID'] + 1
btrack_df['ID_unique'] = btrack_df['ID'] * btrack_df['sample_id_numeric']

# Add shape index
btrack_df['shape_index'] = btrack_df['perimeter'] / np.sqrt(btrack_df['area'])

# Add area_zero and last_area (mean of first/last three timepoint to mitigate segmentation errors)
btrack_df = btrack_df.sort_values(by=['ID_unique', 't'])
def compute_area_zero(group):
    mean_area = group.head(3)['area'].mean()
    group['area_zero'] = mean_area
    return group

def compute_last_area(group):
    mean_area = group.tail(3)['area'].mean()
    group['last_area'] = mean_area
    return group

btrack_df = btrack_df.groupby('ID_unique').apply(compute_area_zero)
btrack_df = btrack_df.groupby('ID_unique').apply(compute_last_area)
btrack_df.reset_index(drop=True, inplace=True)

# Add normalized area
btrack_df['area_norm'] = btrack_df['area'] / btrack_df['area_zero']

# Add last_to_intial_area
btrack_df['last_to_initial_area'] = btrack_df['last_area'] / btrack_df['area_zero']

# Print number of tracks / conditions
unique_tracks = btrack_df['ID_unique'].nunique()
print(f'Unique tracks: {unique_tracks}')
tracks_per_condition = btrack_df.groupby('Condition')['ID_unique'].nunique()
print(f'Tracks per condition {tracks_per_condition}')

In [ ]:
#@title Filter out poorly segmentated/tracked tracks based on area variation

# Function to check if any row in the group has a more than ±50% change
def has_large_area_variation(df):
    df = df.sort_values(by='t')
    df['area_change'] = df['area'].pct_change().fillna(0)
    # Check if any value in 'area_change' exceeds ±50%
    return (df['area_change'].abs() > 0.5).any()

# Filter out groups where any row has a large change
filtered_df = btrack_df.groupby('ID_unique').filter(lambda x: not has_large_area_variation(x))

# Drop the temporary 'area_change' column if needed
filtered_df = filtered_df.drop(columns='area_change', errors='ignore')

# Print the filtered DataFrame
print(filtered_df)

#**UMAP analysis**

In [ ]:
#@title Create UMAP dataframe

# Choose the columns you want to include
columns_to_include = ['orientation', 'minor_axis_length', 'major_axis_length',
                      'area', 'perimeter', 'solidity', 'strain', 'AR',
                      'area_zero', 'AR_zero'
                     ]

# Select relevant columns from btrack_df
selected_df = btrack_df[columns_to_include]

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
btrack_df.sample_id = le.fit_transform(btrack_df.sample_id.values)

# Convert non-numeric values to NaN
selected_df = selected_df.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values
selected_df = selected_df.dropna()

print("Done")

# Display the first few rows of the selected DataFrame
selected_df.head()

In [ ]:
# @title ##Perform UMAP

#@markdown useful UMAP documentation to interpret the parameters:  https://umap-learn.readthedocs.io/en/latest/clustering.html
import umap
import plotly.offline as pyo

#@markdown ###UMAP parameters:

n_neighbors = 15 # @param {type:"raw"}
# n_neighbors = len(selected_df) // 10
min_dist = 0.01  # @param {type: "number"}
# min_dist = 0.5  # @param {type: "number"}



n_dimension = 2


# Initialize UMAP object with the specified settings
reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_dimension, random_state=42)
# Exclude non-numeric columns when fitting UMAP
embedding = reducer.fit_transform(selected_df)
# Create dynamic column names based on n_components
column_names = [f'UMAP_{i}' for i in range(1, n_dimension + 1)]

# # Extract the columns_to_include from selected_df
# included_data = selected_df[columns_to_include].reset_index(drop=True)

# Concatenate the UMAP embedding with ALL the data columns
umap_df = pd.concat([pd.DataFrame(embedding, columns=column_names), btrack_df], axis=1)
# umap_df = pd.concat([pd.DataFrame(embedding, columns=column_names), included_data], axis=1)
#                                                                     ^ or only the included data


In [ ]:
#@title Display UMAP

import matplotlib.pyplot as plt

#@markdown ###Display parameters:
spot_size = 5 # @param {type: "number"}

import warnings

# # Check if the DataFrame has any NaN values and print a warning if it does.
# nan_columns = umap_df.columns[umap_df.isna().any()].tolist()

# if nan_columns:
#   warnings.warn(f"The DataFrame contains NaN values in the following columns: {', '.join(nan_columns)}")
#   for col in nan_columns:
#     umap_df = umap_df.dropna(subset=[col])  # Drop NaN values only from columns containing them

# Visualize the UMAP projection
plt.figure(figsize=(6, 6))

sns.scatterplot(x=column_names[0], y=column_names[1], data=umap_df, palette='Set2', s=spot_size)
plt.title('UMAP Projection of the Dataset')
curr_axis = plt.gca()

# Turn spines off because UMAP numbers dont mean anything
# curr_axis.set_axis_off()
plt.setp(curr_axis.spines.values(), visible=False)
# remove ticks and labels for the left axis
curr_axis.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')


plt.show()


In [ ]:
#@title Color by sample ID and conditions -- see how well mixed observations from each experiemnt are

import matplotlib as mpl

colors_cycler = plt.rcParams['axes.prop_cycle'].by_key()['color']*10


fig, axes = plt.subplots(1,1, figsize=(6,6))
scatter_plot = sns.scatterplot(x='UMAP_1', y='UMAP_2', hue='ID_unique', palette='bright', data=umap_df, s=spot_size, legend = False, ax = axes)
axes.set_title(f'Color by: Track ID (cell)')
scatter_plot.set_xlim(-10,20)
curr_axis = plt.gca()

# Turn spines off because UMAP numbers dont mean anything
# curr_axis.set_axis_off()
plt.setp(curr_axis.spines.values(), visible=False)
# remove ticks and labels for the left axis
curr_axis.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
scatter_plot.set_rasterized(True)


# Setting labels
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')

# Move the axes labels closer to the origin
axes.xaxis.set_label_coords(0.07, -0.03)
axes.yaxis.set_label_coords(-0.03, 0.07)



# with mpl.rc_context({'text.usetex': False,
#                 "svg.fonttype": 'none'
#                 }):
#    plt.savefig("/content/gdrive/MyDrive/UMAP_memGFP_trackID_color_merge.svg", format = 'svg')  # Uncomment to save 2D plot as svg
plt.show()



fig, axes = plt.subplots(1,1, figsize=(6,6))
sns.scatterplot(x='UMAP_1', y='UMAP_2', hue='Condition', palette=colors_cycler, data=umap_df, s=spot_size, legend = True, ax = axes)
axes.set_title(f'Color by: Condition)')
#   plt.savefig("content/Umap/HDBSCAN_clusters_2D.pdf")  # Save 2D plot as PDF
plt.show()


fig, axes = plt.subplots(1,1, figsize=(6,6))
scatter_plot = sns.scatterplot(x='UMAP_1', y='UMAP_2', hue='sample_id', palette='bright', data=umap_df, s=spot_size, legend = True, ax = axes, alpha = 0.5)
# axes.set_title(f'Color by: Sample')
sns.move_legend(axes, "upper left", bbox_to_anchor=(1, 1))
scatter_plot.set_xlim(-10,20)
scatter_plot.set_rasterized(True)
curr_axis = plt.gca()

# Turn spines off because UMAP numbers dont mean anything
# curr_axis.set_axis_off()
plt.setp(curr_axis.spines.values(), visible=False)
# remove ticks and labels for the left axis
curr_axis.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
scatter_plot.set_rasterized(True)


# Setting labels
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')

# Move the axes labels closer to the origin
axes.xaxis.set_label_coords(0.07, -0.03)
axes.yaxis.set_label_coords(-0.03, 0.07)




def find_closest_factors(N):
    # Starting with the square root of N to find the closest factors
    for i in range(int(N**0.5), 0, -1):
        if N % i == 0:
            return i, N // i


def find_nicer_factorization(N):
    while True:
        factors = find_closest_factors(N)

        # Check if the factorization is "nicer", i.e., both factors are closer to each other than just 1 and N
        if np.abs(factors[0] - np.sqrt(N)) <= np.power(N, 0.25)/2:
            return N, factors
        else:
            # Increment N until a nicer factorization is found
            N = N + 1



number_of_unique_samples = umap_df['sample_id'].nunique()

N, factors = find_nicer_factorization(number_of_unique_samples)


fig, axes = plt.subplots(factors[0],factors[1], figsize=(factors[1] * 3,factors[0] * 3), layout = 'constrained')
axes = axes.ravel()

color_palette = sns.color_palette('bright')
for idx_i, sample_id_ in enumerate(umap_df['sample_id'].unique()):
    scatter_plot = sns.scatterplot(x='UMAP_1', y='UMAP_2', color= color_palette[idx_i], alpha=0.5,
                    data=umap_df[umap_df['sample_id'] == sample_id_], s=spot_size, ax=axes[idx_i])
    # axes[idx_i].set_title(f'Sample: {sample_id_}')
    scatter_plot.set_xlim(-10,20)
    scatter_plot.set_rasterized(True)

    # Turn spines off because UMAP numbers dont mean anything
    # curr_axis.set_axis_off()
    plt.setp(scatter_plot.spines.values(), visible=False)
    # remove ticks and labels for the left axis
    scatter_plot.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
    # with mpl.rc_context({'text.usetex': False,
    #                 "svg.fonttype": 'none'
    #                 }):
    #    plt.savefig("/content/gdrive/MyDrive/UMAP_memGFP_sample_color.svg", format = 'svg')  # Uncomment to save 2D plot as PDF

        # Setting labels
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')

    # Move the axes labels closer to the origin
    scatter_plot.xaxis.set_label_coords(0.07, -0.03)
    scatter_plot.yaxis.set_label_coords(-0.03, 0.07)



plt.show()

In [ ]:
#@title Identify clusters using HDBSCAN
import hdbscan
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib as mpl

def mpl_to_plotly(cmap, pl_entries=11, rdigits=2):
    # cmap - colormap
    cmap = mpl.colormaps[cmap]
    # pl_entries - int = number of Plotly colorscale entries
    # rdigits - int -=number of digits for rounding scale values
    scale = np.linspace(0, 1, pl_entries)
    colors = (cmap(scale)[:, :3]*255).astype(np.uint8)
    pl_colorscale = [[round(s, rdigits), f'rgb{tuple(color)}'] for s, color in zip(scale, colors)]
    return pl_colorscale

#@markdown ###HDBSCAN parameters:
clustering_data_source = 'umap'  # @param ['umap', 'raw']
min_samples = 30 # @param ["None"] {type:"raw", allow-input: true}
min_cluster_size = umap_df.shape[0] // 50 # @param {type:"raw"}
# min_cluster_size = umap_df.shape[0] // 50
metric = "euclidean"  # @param ['euclidean', 'manhattan', 'chebyshev', 'braycurtis', 'canberra']

'''
min_cluster_size : int, optional (default=5)
    The minimum size of clusters; single linkage splits that contain
fewer points than this will be considered points "falling out" of a
cluster rather than a cluster splitting into two new clusters.

min_samples : int, optional (default=None)
    The number of samples in a neighbourhood for a point to be
considered a core point.
'''

# Apply HDBSCAN
clusterer = hdbscan.HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size, metric=metric)  # You may need to tune these parameters


In [ ]:
#@title ### Plot the clusters

from matplotlib import patheffects

#@markdown ###Display parameters:
spot_size = 5 # @param {type: "number"}
cmap ='tab20' # @param {type: "string"}
legend_loc = 'on data' #@param {type: "string"}

colors = ["#2f4f4f", "#a0522d", "#6495ed", "#228b22", "#ff69b4",  "#4b0082", "#ff0000", "#ffe119", "#469990", "#00ffff", "#0000ff", "#ff00ff", "#eee8aa"]


if clustering_data_source == 'umap':
    clusterer.fit(umap_df[['UMAP_1', 'UMAP_2']])  # Use two UMAP dims for clustering
else:
    clusterer.fit(selected_df.select_dtypes(include=['number']))

# Add the cluster labels to your UMAP DataFrame
umap_df['Cluster'] = clusterer.labels_

# If the Cluster column already exists in btrack_df, drop it to avoid duplications
if 'Cluster' in btrack_df.columns:
    btrack_df.drop(columns='Cluster', inplace=True)

# Merge the Cluster column from umap_df to btrack_df based on Unique_ID btrack_df = pd.merge btrack_df, umap_df[['Unique_ID', 'Cluster']], on='Unique_ID', how='left')

# Handle cases where some rows in btrack_df might not have a corresponding cluster label btrack_df['Cluster'].fillna(-1, inplace=True)  # Assigning -1 to cells that were not assigned to any cluster

# Save the DataFrame with the identified clusters btrack_df.to_csv(Results_Folder + '/' + 'merged_Tracks.csv', index=False)

# Plotting the results
plt.figure(figsize=(4,4))
scatter_plot = sns.scatterplot(x='UMAP_1', y='UMAP_2', hue='Cluster', palette=colors, data=umap_df, s=spot_size)
plt.title('Clusters Identified by HDBSCAN')
current_ax = plt.gca()

if legend_loc == 'on data':
#### === THIS IS FRON SCANPY === ###
# identify centroids to put labels

    all_pos = (
        pd.DataFrame(umap_df, columns=["UMAP_1", "UMAP_2", "Cluster"])
        .groupby('Cluster', observed=True)
        .median()
        # Have to sort_index since if observed=True and categorical is unordered
        # the order of values in .index is undefined. Related issue:
        # https://github.com/pandas-dev/pandas/issues/25167
        .sort_index()
    )

    for label, x_pos, y_pos in all_pos.itertuples():
        current_ax.text(
            x_pos,
            y_pos,
            label,
            # weight=legend_fontweight,
            verticalalignment="center",
            horizontalalignment="center",
            # fontsize=legend_fontsize,
            path_effects= [patheffects.withStroke(linewidth=2, foreground="w")],
        )
    current_ax.get_legend().remove()
else:
    sns.move_legend(current_ax, "upper left", bbox_to_anchor=(1, 1))

# Turn spines off because UMAP numbers dont mean anything
# curr_axis.set_axis_off()
plt.setp(current_ax.spines.values(), visible=False)
# remove ticks and labels for the left axis
current_ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)

scatter_plot.set_rasterized(True)

# Setting labels
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')

# Move the axes labels closer to the origin
scatter_plot.xaxis.set_label_coords(0.09, -0.03)
scatter_plot.yaxis.set_label_coords(-0.03, 0.11)

# with mpl.rc_context({'text.usetex': False,
#                 "svg.fonttype": 'none'
#                 }):

#    plt.savefig("/content/UMAP_clusters.svg", format = 'svg', dpi = 300)  #



plt.show()



In [ ]:
#@title Plot percentages of conditions in each clusters

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Group the data by Cluster and Condition and calculate the count of each combination
grouped = umap_df.groupby(['Cluster', 'Condition']).size().reset_index(name='Count')

# Pivot the data to have Cluster as columns and Condition as rows
pivot_df = grouped.pivot(index='Cluster', columns='Condition', values='Count')

# Calculate the sum of counts within each Cluster
cluster_sum = pivot_df.sum(axis=1)

# Calculate the percentages within each Cluster
percentage_df = pivot_df.divide(cluster_sum, axis=0) * 100

# Plot the bar chart
x = percentage_df.index
width = 0.5

fig, ax = plt.subplots()

ax.bar(x, percentage_df['memGFP'], width, label='memGFP', color='black')
ax.bar(x, percentage_df['myrRFP'], width, bottom=percentage_df['memGFP'], label='myrRFP', color = 'magenta', alpha=0.65)
ax.bar(x, percentage_df['deAct'], width, bottom=percentage_df['memGFP'] + percentage_df['myrRFP'], label='deAct', color='green')

ax.set_xlabel('Cluster')
ax.set_ylabel('Percentage')
ax.set_title('Percentage of Conditions for Each Cluster')
ax.set_xticks(x)

# Move the legend to the right
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

In [ ]:
#@title ### Plot various features on UMAP space
import matplotlib.pyplot as plt
import seaborn as sns

features_list_to_plot = ['strain', 'area_norm', 'AR', 'ML_zero']

fig, axes = plt.subplots(1, 4, figsize=(22, 4), constrained_layout= False)
plt.subplots_adjust(wspace=0.1)
axes = axes.ravel()

for idx_i, feature_color_by_ in enumerate(features_list_to_plot):
    cutoff_min = np.percentile(umap_df[feature_color_by_], 2.5)
    cutoff_max = np.percentile(umap_df[feature_color_by_], 97.5)

    scatter_plot = sns.scatterplot(x='UMAP_1', y='UMAP_2', hue=umap_df[feature_color_by_], palette='viridis', data=umap_df,
                                    s=spot_size, ax=axes[idx_i], alpha=0.8, legend = False)

    scatter_plot.set_rasterized(True)

    # Refine colormap based on percentiles
    norm = plt.Normalize(cutoff_min, cutoff_max)
    scatter_plot.collections[0].set_norm(norm)
    scatter_plot.collections[0].set_array(umap_df[feature_color_by_])

    plt.setp(scatter_plot.spines.values(), visible=False)
    scatter_plot.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
    plt.xlabel('UMAP 1')
    plt.ylabel('UMAP 2')

    if idx_i == 2:
        plt.colorbar(scatter_plot.collections[0], ax=axes[idx_i], aspect=40, ticks=([1.5, 4.5]))
    elif idx_i == 0:
        plt.colorbar(scatter_plot.collections[0], ax=axes[idx_i], aspect=40, ticks=([0.1, 0.6]))
    elif idx_i == 3:
        colorbar = plt.colorbar(scatter_plot.collections[0], ax=axes[idx_i], aspect=40, ticks=([55.1, 550.7]))
        colorbar.set_ticklabels(['25','250'])

    elif idx_i == 1:
        plt.colorbar(scatter_plot.collections[0], ax=axes[idx_i], aspect=40, ticks=([0.9, 1.4]))



    # with mpl.rc_context({'text.usetex': False,
    #             "svg.fonttype": 'none'
    #             }):
    #     plt.savefig("/content/gdrive/MyDrive/UMAP_strain,area_norm,AR,ML_0.svg", format = 'svg')  # Uncomment to save 2D plot as svg

plt.show()

In [ ]:
#@title Plot initials features for each cluster

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

# Define the features for which you want to create boxplots
features = ['area_zero', 'AR_zero', 'ML_zero']

# Create a colormap
colors = ["#2f4f4f", "#a0522d", "#6495ed", "#228b22", "#ff69b4",  "#4b0082", "#ff0000", "#ffe119", "#469990", "#00ffff", "#0000ff", "#ff00ff", "#eee8aa"]

num_clusters = len(umap_df['Cluster'].unique())

# Create a subplots grid for the boxplots
fig, axes = plt.subplots(nrows=len(features), ncols=1, figsize=(7, 16))

# Iterate over each feature and create a boxplot in a subplot
for i, feature in enumerate(features):

    # Extract the first occurrence for each cell
    first_occurrence = umap_df.groupby('ID_unique')[feature].first().reset_index()

    # Merge with umap_df to include the "Cluster" column
    first_occurrence = first_occurrence.merge(umap_df[['ID_unique', 'Cluster']], on='ID_unique')

    first_occurrence = first_occurrence.drop_duplicates()

    # Create a combined plot with boxplot and stripplot
    boxplot = sns.boxplot(data=first_occurrence, x='Cluster', y=feature, hue='Cluster', showfliers=False, palette=colors, ax=axes[i], legend = False)
    sns.stripplot(data=first_occurrence, x='Cluster', y=feature, hue='Cluster', jitter=True, alpha=0.2, palette=colors, edgecolor='black', linewidth=1, ax=axes[i], legend = False)

    boxplot.set_title(f'Boxplot of {feature} by Cluster')
    boxplot.set_xlabel('Cluster')
    boxplot.set_ylabel(feature)


    # Pixel to micron correction
    yticklabels_ = np.arange(200, 701, 100, )
    axes[0].set_yticks([i_ * (2.205**2) for i_ in yticklabels_])
    axes[0].set_yticklabels([str(label) for label in yticklabels_])

    yticklabels_ = np.arange(50, 351, 50, )
    axes[1].set_yticks([i_ * 2.205 for i_ in yticklabels_])
    axes[1].set_yticklabels([str(label) for label in yticklabels_])


# Adjust the spacing between subplots
plt.tight_layout()

# Save the figure
# with mpl.rc_context({'text.usetex': False, "svg.fonttype": 'none'}):
#     plt.savefig("/content/gdrive/MyDrive/Clusters_initial_features_boxplots.svg", format='svg') # Uncomment to save 2D plots as svg

# Show the figure
plt.show()

In [ ]:
#@title Plot various features vs strain for each cluster
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline

METRIC_COLUMN_NAME = ['area_norm', 'AR', 'orientation']

# Group by 'ID_unique' and filter

def cubic_interpolation_no_duplicates(group, num_points=200):
    group = group.sort_values(by='strain')  # Sort by strain
    group = group.drop_duplicates(subset='strain')  # Remove duplicates
    x = group['strain']
    y = group[metric]

    f = interp1d(x, y,
                 kind='linear',   # Use linear interpolation, vs 'cubic'
                 fill_value=('nan'),
                 bounds_error=False,
                 assume_sorted=True)

    new_strains = np.linspace(0, MIN_FINAL_STRAIN, num_points)
    new_area_norms = f(new_strains)
    return pd.Series(new_area_norms)

MIN_FINAL_STRAIN = 0.7

# Get the number of unique clusters
num_clusters = len(umap_df['Cluster'].unique())

# Create a colormap with 'num_clusters' colors
colors = ["#2f4f4f", "#a0522d", "#6495ed", "#228b22", "#ff69b4",  "#4b0082", "#ff0000", "#ffe119", "#469990", "#00ffff", "#0000ff", "#ff00ff", "#eee8aa"]

rolling_window = 10
# Create a separate figure
fig, axs = plt.subplots(len(METRIC_COLUMN_NAME), num_clusters, figsize=((3 * num_clusters), (3 * len(METRIC_COLUMN_NAME))))

# Iterate over each cluster
unique_clusters = np.sort(umap_df['Cluster'].unique())

for j, cluster in enumerate(unique_clusters):
    # Iterate over each metric column
    for i, metric in enumerate(METRIC_COLUMN_NAME):
        # Get the corresponding subplot
        ax = axs[i, j]

        cluster_df = umap_df[umap_df['Cluster'] == cluster]
        cluster_df = cluster_df[['ID_unique', 'strain', metric]]
        cluster_df = cluster_df.groupby('ID_unique').filter(lambda x: x['strain'].max() > MIN_FINAL_STRAIN)

        # Filter out groups with less than 3 values for strain
        group_counts = cluster_df.groupby('ID_unique').size()
        valid_groups = group_counts[group_counts > 3].index
        cluster_df = cluster_df[cluster_df['ID_unique'].isin(valid_groups)]

        interpolated_data = cluster_df.groupby('ID_unique').apply(cubic_interpolation_no_duplicates)

        final_df = interpolated_data
        final_df.columns.name = None
        final_df.reset_index(drop=True, inplace=True)

        mean_values = final_df.mean(axis=0)
        std_values = final_df.std(axis=0)

        # Apply rolling average to smooth the curves
        mean_values_smoothed = mean_values.rolling(rolling_window, center=True).mean()
        std_values_smoothed = std_values.rolling(rolling_window, center=True).mean()


        strain_values = np.linspace(0, MIN_FINAL_STRAIN, final_df.shape[-1])

        color = colors[j]
        ax.plot(strain_values, mean_values_smoothed, color=color, label=f'Cluster {cluster}')
        ax.fill_between(strain_values, mean_values_smoothed - std_values_smoothed, mean_values_smoothed + std_values_smoothed,
                        color=color, alpha=0.3)

        # Set the labels and title for the subplot
        ax.set_xlabel('Strain')
        ax.set_ylabel(f'Mean {metric}')
        ax.set_title(f'Cluster {cluster}: Mean {metric}')
        ax.set_xlim(0,0.7)
        if i == 0:  # First subplot
            ax.set_ylim(0.9, 1.5)
        elif i == 1:  # Second subplot
            ax.set_ylim(1, 5)
        elif i == 2:  # Third subplot
            ax.set_ylim(-1.5, 1.5)


        # Add a legend to the subplot
        ax.legend()

# Adjust the spacing between subplots
plt.tight_layout()

# with mpl.rc_context({'text.usetex': False,
#                 "svg.fonttype": 'none'
#                 }):
#     plt.savefig("/content/gdrive/MyDrive/Cluster_featuresVSstrain.svg", format = 'svg') # Uncomment to save 2D plot as svg

# Show the plot
plt.show()

In [ ]:
#@title Plot intials features of specific clusters

import seaborn as sns
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

# Define the features for which you want to create boxplots
features = ['area_zero', 'AR_zero', 'ML_zero']

# Define the clusters you want to plot
clusters_to_plot = [1, 3]

colors = ["#2f4f4f", "#a0522d", "#6495ed", "#228b22", "#ff69b4",  "#4b0082", "#ff0000", "#ffe119", "#469990", "#00ffff", "#0000ff", "#ff00ff", "#eee8aa"]

# Create a colormap with 'num_clusters' colors
color_map = [colors[cluster + 1] for cluster in clusters_to_plot]

# Custom order for the clusters
custom_order = [1, 3]

# Create a single plot with subplots for each feature
fig, axes = plt.subplots(len(features), figsize=(5, 5 * len(features)))

# Iterate over each feature and create a subplot
for i, feature in enumerate(features):
    ax = axes[i]

    # Extract the first occurrence for each cell
    first_occurrence = umap_df.groupby('ID_unique')[feature].first().reset_index()

    # Merge with umap_df to include the "Cluster" column
    first_occurrence = first_occurrence.merge(umap_df[['ID_unique', 'Cluster']], on='ID_unique')

    first_occurrence = first_occurrence.drop_duplicates()

    # Filter the data for only the specified clusters
    filtered_data = first_occurrence[first_occurrence['Cluster'].isin(clusters_to_plot)].copy()

    # Convert 'Cluster' column to categorical variable with custom order
    cluster_order = CategoricalDtype(categories=custom_order, ordered=True)
    filtered_data.loc[:, 'Cluster'] = filtered_data['Cluster'].astype(cluster_order)

    # Create a combined plot with boxplot and stripplot
    sns.boxplot(data=filtered_data, x='Cluster', y=feature, hue='Cluster', showfliers=False, palette=color_map, ax=ax)
    sns.stripplot(data=filtered_data, x='Cluster', y=feature, hue='Cluster', jitter=True, alpha=0.2, palette=color_map, edgecolor='black', linewidth=1, ax=ax, legend=False)

    ax.set_title(f'Boxplot of {feature} for Clusters {clusters_to_plot}')
    ax.set_xlabel('Cluster')
    ax.set_ylabel(feature)

with mpl.rc_context({'text.usetex': False,
                "svg.fonttype": 'none'
                }):
    plt.savefig("/content/gdrive/MyDrive/Specific_clusters_initial_features.svg", format = 'svg')  # Save 2D plot as PDF
plt.show()


In [ ]:
#@title Plot various features vs strain for specific clusters

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from scipy.interpolate import interp1d
import matplotlib as mpl


METRIC_COLUMN_NAME = ['area_norm', 'AR']

# Group by 'ID_unique' and filter
def cubic_interpolation_no_duplicates(group, num_points=200):
    group = group.sort_values(by='strain')  # Sort by strain
    group = group.drop_duplicates(subset='strain')  # Remove duplicates
    f = interp1d(group['strain'], group[metric],
                 kind='linear',  # cubic
                 bounds_error=False, fill_value="nan")
    new_strains = np.linspace(0, MIN_FINAL_STRAIN, num_points)
    new_area_norms = f(new_strains)
    return pd.Series(new_area_norms)

MIN_FINAL_STRAIN = 0.7


# Get the number of unique clusters
num_clusters = len(umap_df['Cluster'].unique())

# Create a separate figure
fig, ax = plt.subplots(len(METRIC_COLUMN_NAME), 1, figsize=(3, 6), layout = 'constrained')

# List of desired clusters
desired_clusters = [[1], [3], [1,2,3,4,5,6,7,8,9,10,11]]
MAX_FINAL_STRAIN = umap_df.loc[umap_df['Cluster'].isin(desired_clusters), 'strain'].max()


colors = ["#2f4f4f", "#a0522d", "#6495ed", "#228b22", "#ff69b4",  "#4b0082", "#ff0000", "#ffe119", "#469990", "#00ffff", "#0000ff", "#ff00ff", "#eee8aa"]
color_map = [colors[cluster[-1] - 1] for cluster in desired_clusters]

# Rolling average window size
rolling_window = 10

# Iterate over each metric column
for i, metric in enumerate(METRIC_COLUMN_NAME):
    for idx_i, cluster in enumerate(desired_clusters):
        cluster_df = umap_df[umap_df['Cluster'].isin(cluster)]
        cluster_df = cluster_df[['ID_unique', 'strain', metric]]
        cluster_df = cluster_df.groupby('ID_unique').filter(lambda x: x['strain'].max() > MIN_FINAL_STRAIN)

        # Filter out groups with less than 3 values for strain
        group_counts = cluster_df.groupby('ID_unique').size()
        valid_groups = group_counts[group_counts > 3].index
        cluster_df = cluster_df[cluster_df['ID_unique'].isin(valid_groups)]

        interpolated_data = cluster_df.groupby('ID_unique').apply(cubic_interpolation_no_duplicates)

        final_df = interpolated_data
        final_df.columns.name = None
        final_df.reset_index(drop=True, inplace=True)

        mean_values = final_df.mean(axis=0)
        std_values = final_df.std(axis=0)

        # Apply rolling average to smooth the curves
        mean_values_smoothed = mean_values.rolling(rolling_window, center=True).mean()
        std_values_smoothed = std_values.rolling(rolling_window, center=True).mean()

        strain_values = np.linspace(0, MIN_FINAL_STRAIN, final_df.shape[-1])

        hatch_ = ''
        if idx_i == 3:
            hatch_ = '/'

        color = colors[cluster[-1]+1]
        ax[i].plot(strain_values, mean_values_smoothed, color=color, label=f'Cluster {cluster[-1]}')
        ax[i].fill_between(strain_values, mean_values_smoothed - std_values_smoothed, mean_values_smoothed + std_values_smoothed,
                        color=color, alpha=0.4, hatch = hatch_)

    # Set the labels and title for the subplot
    ax[i].set_xlabel('Strain')
    ax[i].set_ylabel(f'Mean {metric}')
    # ax[i].set_title(f'Mean {metric} by cluster')
    ax[i].set_xlim(-0.01,0.7)

    # Add a legend to the subplot
    ax[i].legend()


with mpl.rc_context({'text.usetex': False,
                    "svg.fonttype": 'none'
                    }):
    plt.savefig("/content/Specific_clusters_featuresVSstrain.svg", format='svg')  # Save 2D plot as SVG

# Show the plot
plt.show()

In [ ]:
#@title Plot max normalized area VS initial area

import matplotlib.pyplot as plt

import pandas as pd

# Assuming 'umap_df' is the pandas DataFrame containing the data

colors = ["#2f4f4f", "#a0522d", "#6495ed", "#228b22", "#ff69b4",  "#4b0082", "#ff0000", "#ffe119", "#469990", "#00ffff", "#0000ff", "#ff00ff", "#eee8aa"]

# Sort the DataFrame by 'Cluster' column
umap_df.sort_values(by='Cluster', inplace=True)

area_norm_max = []
area_zero = []

# Group the DataFrame by 'ID_unique' column and calculate the maximum 'area_norm' within each group
grouped_df = umap_df.groupby('ID_unique')['area_norm'].max()

# Track the clusters for which the legend labels have been added
added_clusters = set()

# Iterate over unique clusters
for ID_unique in umap_df['ID_unique'].unique():
    # Append the maximum 'area_norm' value for the current cluster
    area_norm_max.append(grouped_df.loc[ID_unique])

    # Retrieve the first 'area_zero' value for the current cluster
    ID_unique_area_zero = umap_df.loc[umap_df['ID_unique'] == ID_unique, 'area_zero'].iloc[0]
    area_zero.append(ID_unique_area_zero)

    # Assign color to the current cluster
    cluster = umap_df.loc[umap_df['ID_unique'] == ID_unique, 'Cluster'].iloc[0]
    color_idx = cluster + 1  # Adjust index to account for clusters starting from -1
    color = colors[color_idx]

    # Only add the legend label for the cluster if it has not been added before
    if cluster not in added_clusters:
        plt.scatter(ID_unique_area_zero, grouped_df.loc[ID_unique], s=5, color=color, label=f"Cluster {cluster}")
        added_clusters.add(cluster)
    else:
        plt.scatter(ID_unique_area_zero, grouped_df.loc[ID_unique], s=5, color=color, alpha = 0.75)

# Set labels for the x-axis and y-axis
plt.xlabel('area_zero')
plt.ylabel('max_area_norm')

# plt.xticks([970.3, 1455.5, 1940.7, 2425.8, 2911, 3396], ['200', '300', '400', '500', '600', '700'])

# Set a title for the plot
plt.title('max_area_norm vs area_zero')

# Add legend
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Save the figure
with mpl.rc_context({'text.usetex': False, "svg.fonttype": 'none'}):
    plt.savefig("/content/gdrive/MyDrive/max_area_norm VS area_zero.svg", format='svg')

# Display the plot
plt.show()

#**Plot average conditions per conditions**

In [ ]:
#@title Plot intials features per condition
import seaborn as sns
import matplotlib.pyplot as plt

# Define the features for which you want to create boxplots
features = ['area_zero', 'AR_zero', 'ML_zero']

# Create a colormap with 'num_clusters' colors
color_map = ['lightgrey', 'darkgreen', 'dimgrey', 'lightgreen']
condition_order = ['memGFP', 'myrRFP (from memGFP)', 'deActGS1', "myrRFP (from deActGS1)"]
palette = dict(zip(condition_order, color_map))  # Map conditions to colors

# Define hatch patterns for each condition
hatch_dict = {
    'memGFP': '//',
    'myrRFP (from memGFP)': '',
    'deActGS1': '',
    'myrRFP (from deActGS1)': '//'
}

# Create a new figure and subplots with the desired layout
fig, axes = plt.subplots(3, 1, figsize=(7, 20))

# Create a separate boxplot for each feature
for i, feature in enumerate(features):
    # Extract the first occurrence for each cell
    first_occurrence = filtered_subset_df.groupby('ID_unique')[feature].first().reset_index()

    # Merge with umap_df to include the "Cluster" column
    first_occurrence = first_occurrence.merge(filtered_subset_df[['ID_unique', 'Condition']], on='ID_unique')

    first_occurrence = first_occurrence.drop_duplicates()

    # Create a combined plot with boxplot and stripplot
    boxplot = sns.boxplot(data=first_occurrence, x='Condition', y=feature, hue='Condition',
                          showfliers=False, palette=color_map, legend=False,
                          order=condition_order, ax=axes[i])
    sns.stripplot(data=first_occurrence, x='Condition', y=feature, hue='Condition',
                  jitter=0.1, alpha=0.2, palette=color_map, edgecolor='black',
                  linewidth=1, legend=False, ax=axes[i],
                          dodge = False)

    axes[i].set_title(f'Boxplot of {feature} by Condition')
    axes[i].set_xlabel('Condition')
    axes[i].tick_params(axis='x', rotation=30)
    axes[i].set_ylabel(feature)
    axes[0].set_yticks([0,970.3,1940.7,2911,3881.3, 4851.7, 5822])
    axes[0].set_yticklabels(['0','200', '400', '600','800','1000', '1200'])

        # Step 4: Apply hatches to boxplot
    num_conditions = len(condition_order)
    for i, box in enumerate(boxplot.patches):
        condition_index = i % num_conditions
        condition = condition_order[condition_index]
        hatch = hatch_dict[condition]
        if hatch:
            box.set_hatch(hatch)

# Adjust the spacing between subplots
fig.tight_layout()

with mpl.rc_context({'text.usetex': False,
                "svg.fonttype": 'none'
                }):
    plt.savefig("/content/gdrive/MyDrive/Condition_initial_features.svg", format = 'svg')  # Save 2D plot as PDF

# Show the figure
plt.show()


In [ ]:
#@title Perform ANOVA on intials features per condition

import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

# Define the feature to analyze
feature = 'area_zero'

# Extract the first occurrence for each ID and merge with condition
first_occurrence = filtered_df.groupby('ID_unique')[feature].first().reset_index()
first_occurrence = first_occurrence.merge(filtered_df[['ID_unique', 'Condition']], on='ID_unique')
first_occurrence = first_occurrence.drop_duplicates()

# Split data by condition
groups = first_occurrence.groupby('Condition')[feature]

# Perform one-way ANOVA
fvalue, pvalue = stats.f_oneway(*[group for name, group in groups])
print(f"ANOVA p-value for {feature}: {pvalue}")

# Tukey's HSD test
tukey_results = pairwise_tukeyhsd(first_occurrence[feature], first_occurrence['Condition'])
print(tukey_results)

# Sample sizes
sample_sizes = first_occurrence['Condition'].value_counts()
print("\nSample Sizes:\n", sample_sizes)

In [ ]:
#@title Perform linear interpolation of chosen feature for each condition
from scipy.interpolate import interp1d

METRIC_COLUMN_NAME = 'area_norm'
MIN_FINAL_STRAIN = 0.7

condition_df_list = []

for condition_ in list(filtered_df['Condition'].unique()):
    condition = filtered_df[filtered_df['Condition'] == condition_].copy()
    condition = condition[['sample_id','ID_unique', 'strain', METRIC_COLUMN_NAME,]]

    # condition.head()
    print(condition_)
    print(condition.shape)

    # Convert non-numeric values to NaN
    condition = condition.apply(pd.to_numeric, errors='coerce')

    # Group by 'ID_unique' and filter
    filtered_groups = condition.groupby('ID_unique').filter(lambda x: x['strain'].max() > MIN_FINAL_STRAIN)

    def linear_interpolation_no_duplicates(group, num_points=200):
        group = group.sort_values(by='strain')  # Sort by strain
        group = group.drop_duplicates(subset='strain')  # Remove duplicates

        f = interp1d(group['strain'], group[METRIC_COLUMN_NAME],
                    kind='linear',
                    bounds_error=False, fill_value="nan")

        new_strains = np.linspace(0, MIN_FINAL_STRAIN, num_points)
        new_area_norms = f(new_strains)

        return pd.Series(new_area_norms)

    # Apply linear interpolation to each group
    interpolated_data = filtered_groups.groupby('ID_unique').apply(linear_interpolation_no_duplicates)

    # Transpose and reset index to get the desired format
    final_df = interpolated_data
    final_df.columns.name = None
    final_df.reset_index(drop=True, inplace=True)

    # Convert non-numeric values to NaN
    final_df = final_df.apply(pd.to_numeric, errors='coerce')

    # Drop rows with NaN values
    # final_df = final_df.dropna()

    print(final_df.shape)
    # final_df.head()  # Display the first few rows of the final dataframe

    condition_df_list.append(final_df)

unique_sample_counts = btrack_df.groupby('Condition')['sample_id'].nunique()
print(unique_sample_counts)

In [ ]:
#@title Plot tracks and average behavior VS strain per conditions

from scipy.signal import savgol_filter

strain_values = np.linspace(0, MIN_FINAL_STRAIN, final_df.shape[-1])

color_map = ['dimgray', 'darkgreen', 'black', 'limegreen']
condition_order = ['myrRFP (from memGFP)', 'deActGS1', 'memGFP', "myrRFP (from deActGS1)"]

unique_sample_counts = btrack_df.groupby('Condition')['sample_id'].nunique()
ordered_counts = unique_sample_counts.reindex(condition_order, fill_value=0)
n_exp = ordered_counts.values

conditions_labels = condition_order
print(conditions_labels)

# Plotting each row (each ID_unique's interpolated data) as a separate line plot
plt.figure(figsize=(4, 4))

for final_df in condition_df_list:
    for index, row in final_df.iterrows():
        row_data =  np.array(row)
        plt.plot(strain_values, row_data,
                #  label=f'ID {int(row)}',
                #  marker = 'o', linestyle = ''
                alpha = 0.3,
                )

    plt.xlabel('Strain')
    plt.ylabel(METRIC_COLUMN_NAME)
    # plt.ylim(bottom = 0)
    plt.title(f'# of Tracks  = {index})')
    # plt.legend()
plt.show()


plt.figure(figsize=(4, 4))

# Parameters for the Savitzky-Golay filter
window_size = 11  # Must be an odd integer
poly_order = 1    # Polynomial order, should be less than window_size

hatch_patterns = ['//', '', '', '//']  # Define hatch patterns for the dim colors

for idx_i, final_df in enumerate(condition_df_list):

    # Calculate the rolling average
    mean_values = final_df.mean()
    mean_values_smooth = savgol_filter(mean_values, window_size, poly_order)
    std_values = final_df.std(axis=0)
    sem_values = std_values / np.sqrt(n_exp[idx_i])
    sem_values_smooth = savgol_filter(sem_values, window_size, poly_order)

    # Plot the smoothed data
    plt.plot(strain_values, mean_values_smooth,
             color=color_map[idx_i],
             label=condition_order[idx_i],
             alpha=0.95)

    # Add hatched filling for dim colors using the hatch parameter
    plt.fill_between(strain_values,
                     mean_values_smooth - sem_values_smooth,
                     mean_values_smooth + sem_values_smooth,
                     color=color_map[idx_i],
                     alpha=0.2,
                     hatch=hatch_patterns[idx_i])

plt.xlabel('Strain')
plt.ylabel(METRIC_COLUMN_NAME)
plt.legend(fontsize=12, loc='upper left')

# with mpl.rc_context({'text.usetex': False,
#                 "svg.fonttype": 'none'
#                 }):
#     plt.savefig("/content/gdrive/MyDrive/Condition_ARVSstrain.svg", format = 'svg')  # Uncomment to save 2D plot as svg

plt.show()


In [ ]:
#@title Perform ANCOVA to test for differences in average behavior between conditions
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('strain ~ C(Condition, Treatment(reference="myrRFP (from deActGS1)")) + area_norm', filtered_df).fit()
print(model.summary())

ancova_results = sm.stats.anova_lm(model, typ=2)
print(ancova_results)

In [ ]:
#@title Plot Last area/initial area per condition

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define custom color palette and condition order
color_map = ['dimgray', 'lightgray', 'darkgreen', 'lightgreen']
condition_order = ['memGFP', 'myrRFP (from memGFP)', 'deActGS1', "myrRFP (from deActGS1)"]
palette = dict(zip(condition_order, color_map))  # Map conditions to colors

# Define hatch patterns for each condition
hatch_dict = {
    'memGFP': '',
    'myrRFP (from memGFP)': '//',
    'deActGS1': '',
    'myrRFP (from deActGS1)': '//'
}

# Step 1: Ensure only one value per 'ID_unique' and 'Condition'
# Group by 'ID_unique' and calculate the mean for each 'Condition'
unique_values_df = filtered_df.groupby(['ID_unique', 'Condition'], as_index=False).agg({'last_to_initial_area': 'mean'})

# Step 2: Create boxplot
plt.figure(figsize=(10, 12))
sns_boxplot = sns.boxplot(
    data=unique_values_df,  # Use unique values
    x='Condition',
    y='last_to_initial_area',
    order=condition_order,  # Set the specific order
    palette=palette,
    showfliers=False
)

# Step 3: Add individual points on top of the boxplot with matching colors
sns.stripplot(
    data=unique_values_df,  # Use unique values
    x='Condition',
    y='last_to_initial_area',
    order=condition_order,
    palette=palette,
    jitter=0.1,
    alpha=0.5,  # Slightly higher transparency for better visibility
    edgecolor='black',
    linewidth=1,
    size=8
)

# Step 4: Apply hatches to boxplot
num_conditions = len(condition_order)
for i, box in enumerate(sns_boxplot.patches):
    condition_index = i % num_conditions
    condition = condition_order[condition_index]
    hatch = hatch_dict[condition]
    if hatch:
        box.set_hatch(hatch)

# Step 5: Customize plot
plt.ylabel('Last Area / Initial Area')
plt.title('Last Area / Initial Area Ratio Across Conditions')
plt.grid(axis='y', linestyle='--', linewidth=0.5)
plt.tight_layout()

# with mpl.rc_context({'text.usetex': False,
#                 "svg.fonttype": 'none'
#                 }):
#     plt.savefig("/content/gdrive/MyDrive/Condition_A_f_to_A_0.svg", format = 'svg')  # Uncomment to save 2D plot as PDF

# Step 6: Display plot
plt.show()

In [ ]:
#@title Perform ANOVA on last/intial areas

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Step 1: Perform One-Way ANOVA
anova_results = stats.f_oneway(
    *[unique_values_df[unique_values_df['Condition'] == condition]['last_to_initial_area'] for condition in condition_order]
)

print("One-way ANOVA result:")
print(f"F-statistic: {anova_results.statistic}, p-value: {anova_results.pvalue}")

# If ANOVA is significant (p-value < 0.05), proceed with post-hoc tests

# Step 2: Perform Post-hoc Tukey's HSD test
# Fit the model
model = ols('last_to_initial_area ~ C(Condition)', data=unique_values_df).fit()

# Perform ANOVA for checking significance (using statsmodels)
anova_table = anova_lm(model)
print("\nANOVA Table:")
print(anova_table)

# Post-hoc Tukey's HSD test
tukey = pairwise_tukeyhsd(endog=unique_values_df['last_to_initial_area'],
                          groups=unique_values_df['Condition'],
                          alpha=0.05)

# Print Tukey's HSD test result
print("\nPost-hoc Tukey HSD Test Results:")
print(tukey.summary())

# # Print the number of samples (n) for each condition
# n_per_condition = merged_df.groupby('Condition')['last_to_initial_area'].count()
# print("\nNumber of samples (n) per condition:")
# print(n_per_condition)